In [1]:
import tiktoken

### 2.5 Byte pair encoding

In [2]:
tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
"of someunknownPlace. し"

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

tokenizer.decode(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114]


'Hello, do you like tea? <|endoftext|> In the sunlit terraces'

### 2.6 Data sampling with a sliding window

In [4]:
with open("the-verdict.txt","r",encoding="utf-8") as fp:
    raw_text = fp.read()

enc_text = tokenizer.encode(raw_text)

enc_sample = enc_text[50:] #remove first 50 tokens

context_size = 4
x = enc_sample[:context_size] #0-4
y = enc_sample[1:context_size+1] #1-5

print(f"x: {x}")
print(f"y:     {y}")


x: [290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [5]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desire = enc_sample[i]
    print(context, "--->", desire)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


In [6]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desire = enc_sample[i]
    print(tokenizer.decode(context), "--->", tokenizer.decode([desire]))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]




In [8]:
dataset = GPTDatasetV1(raw_text,tokenizer,5,1)
dataset[0]

(tensor([  40,  367, 2885, 1464, 1807]),
 tensor([ 367, 2885, 1464, 1807, 3619]))

In [9]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader


In [10]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()


dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [11]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,shuffle=False
)
data_iter = iter(dataloader)
inputs, target = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", target)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### Creating token embedding

In [12]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [13]:
input_ids = torch.tensor([2,3,5,1])
embedding_layer(input_ids)

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

### Encoding word positions

In [14]:
output_dim = 256
vocab_size = 50257

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=4,shuffle=False
)

inputs, target = next(iter(dataloader))

print("Input shape:\n", inputs.shape)

Input shape:
 torch.Size([8, 4])


In [15]:
token_embedding = token_embedding_layer(inputs)
print(token_embedding.shape)

torch.Size([8, 4, 256])


In [16]:
torch.arange(4)

tensor([0, 1, 2, 3])

In [17]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length))

pos_embedding.shape

torch.Size([4, 256])

In [18]:
inputs_embedding = pos_embedding + token_embedding
inputs_embedding.shape

torch.Size([8, 4, 256])

## 3.1 Coding attention mechanism

In [19]:

inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your(x^1)
[0.55, 0.87, 0.66], # journey(x^2)
[0.57, 0.85, 0.64], # starts(x^3)
[0.22, 0.58, 0.33], # with(x^4)
[0.77, 0.25, 0.10], # one(x^5)
[0.05, 0.80, 0.55]] # step(x^6)
)

1. Compute attention score of x^2, which is the dot products of query, x^2 with every other input token
可以看作是x^2 和其他vector 的相似度

In [20]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0]) #the number of token
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) #calculate the dot products between query and each token embedding

print(attn_scores_2)


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


2. Normalize attention score to get attention weight that sum to 1

In [22]:
attention_weight_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attention_weight_2_tmp)
print("Sum:", attention_weight_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [ ]:
attn_weight_2 = torch.softmax(attn_scores_2,dim=0)   #in practice, it is common to use softmax function for normalization

print("Attention weights:", attn_weight_2)
print("Sum:", attn_weight_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


3. Calculate context vector z^2 by multiplying the embedded input tokens, x^i, with the corresponding weights and then summing the resulting vectors
weight乘与对应vector再加在一起


In [30]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weight_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.2 Computing attention weights for all input tokens

1. Compute attention scores (**dot product/similarity** between query and each input) length is the sequence length
2. Compute attention weights (**normalized** version of attention score)
3. Compute context vector (**weight sum** over the inputs)

Compute attention scores for all inputs

In [31]:
attn_scores = torch.empty(6, 6) #6 inputs for each input has 6 attention scores
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i, x_j)
print(attn_scores)


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
